In [48]:
import numpy as np
import matplotlib.pyplot as plt 
import torch
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
from classes import WikiArts
import pandas as pd
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

from torch.utils.data import DataLoader

In [49]:
path_labels = r'./output/path_label.csv'
path_images = r'./wikiart_500_paintings'

In [83]:
def confusion_matrix_bis(outputs_true, outputs_pred):
    '''
    Input: 
        outputs_true: pytorch tensor
        outputs_pred: pytorch tensor
    '''

    path_label_artists = pd.read_csv('./output/path_label_artists.csv')
    to_use = path_label_artists[['artist', 'label']].drop_duplicates()
    dic = to_use.set_index('label')['artist'].to_dict()

    outputs_true = [int(x) for x in sum([list(x) for x in outputs_true], [])]
    outputs_pred  = [int(x) for x in sum([list(x) for x in outputs_pred], [])]
    outputs_true_artists = list(map(dic.get, outputs_true))
    outputs_pred_artists  = list(map(dic.get, outputs_pred))

    plt.rcParams["figure.figsize"] = (50,10)
    cm = confusion_matrix(outputs_true_artists, outputs_pred_artists, labels = list(dic.values()), normalize = 'true')
    disp = ConfusionMatrixDisplay(
        confusion_matrix=np.round(cm, 2), display_labels = list(dic.values())
    )
    disp.plot(xticks_rotation= 45.0)

Pre processing images with new split

In [54]:
transform_normalized = transforms.Compose([transforms.ToPILImage(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

transform_unnormalized = transforms.Compose([transforms.ToPILImage(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
        
data_normalized = WikiArts(path_labels, path_images, transform_normalized)
data_unnormalized = WikiArts(path_labels, path_images, transform_unnormalized)

In [8]:
train_dataset_normalized, test_dataset_normalized = torch.utils.data.random_split(data_normalized, [len(data_normalized) - int(0.15*len(data_normalized)), int(0.15*len(data_normalized))])
train_dataset_normalized, valid_dataset_normalized = torch.utils.data.random_split(train_dataset_normalized, [len(train_dataset_normalized) - int(0.15*len(data_normalized)), int(0.15*len(data_normalized))])

In [9]:
train_dataset_unnormalized, test_dataset_unnormalized = torch.utils.data.random_split(data_unnormalized, [len(data_unnormalized) - int(0.15*len(data_unnormalized)), int(0.15*len(data_unnormalized))])
train_dataset_unnormalized, valid_dataset_unnormalized = torch.utils.data.random_split(train_dataset_unnormalized, [len(train_dataset_unnormalized) - int(0.15*len(data_unnormalized)), int(0.15*len(data_unnormalized))])

Pre processing images with data loader

In [11]:
test_set_method1 = torch.load('./fixing_issues/test_loader_basic_no_mom.pth')
test_set_method2 = torch.load('./fixing_issues/test_loader_basic_with_mom.pth')
test_set_method3 = torch.load('./fixing_issues/XXXXX.pth')
test_set_method4 = torch.load('./fixing_issues/test_loader_lay6and7.pth')
test_set_method5 = torch.load('./fixing_issues/test_loader_w_2_transf.pth')
test_set_method6 = torch.load('./fixing_issues/test_loader_w_4trasnf.pth')


AttributeError: Can't get attribute '_unpickle_block' on <module 'pandas._libs.internals' from '/home/hub/.local/lib/python3.8/site-packages/pandas/_libs/internals.cpython-38-x86_64-linux-gnu.so'>

# Method 1

Testing on loader

In [70]:
path_model = r'./fixing_issues/best_model_optmiSGD_lossCrossEntrop_dav_basic_no_mom.pth'

models = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_resnet50', pretrained=True)
num_classes, batch_size = 23, 25
models.fc = torch.nn.Linear(models.fc.in_features, num_classes)
models.load_state_dict(torch.load(path_model))

optimizers = optim.SGD(models.parameters(), lr=0.001)

Using cache found in /home/hub/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


In [72]:
models.eval()
outputs_pred = []
outputs_true = []
with torch.no_grad(): 
    test_error_count = 0.0
    for images, labels in iter(test_set_method1):
        outputs = models(images)
        outputs_true.append(labels)
        outputs_pred.append(outputs.argmax(1))
        test_error_count += float(len(labels[labels != outputs.argmax(1)]))
    test_accuracy = 1.0 - float(test_error_count) / float(len(test_set_method1)*batch_size)
    print('accuracy test set: %f' % (test_accuracy))

accuracy test set: 0.789643


In [84]:
confusion_matrix_bis(outputs_true, outputs_pred)

TypeError: confusion_matrix() got an unexpected keyword argument 'labels'

Testing on new split

In [ ]:
path_model = r'./fixing_issues/best_model_optmiSGD_lossCrossEntrop_dav_basic_no_mom.pth'
models = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_resnet50', pretrained=True)
num_classes, batch_size = 23, 25
models.fc = torch.nn.Linear(models.fc.in_features, num_classes)
models.load_state_dict(torch.load(path_model))

optimizers = optim.SGD(models.parameters(), lr=0.001)

In [ ]:
models.eval()
outputs_pred = []
outputs_true = []
with torch.no_grad(): 
    test_error_count = 0.0
    for images, labels in iter(test_dataset_unnormalized):
        outputs = models(images)
        outputs_true.append(labels)
        outputs_pred.append(outputs.argmax(1))
        test_error_count += float(len(labels[labels != outputs.argmax(1)]))
    test_accuracy = 1.0 - float(test_error_count) / float(len(test_dataset_unnormalized)*batch_size)
    print('accuracy test set: %f' % (test_accuracy))

In [ ]:
confusion_matrix_bis(outputs_true, outputs_pred)

# Method 2

Testing on loader

In [ ]:
path_model = r'best_model_optmiSGD_lossCrossEntrop_dav_basic_with_mom.pth'
models = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_resnet50', pretrained=True)
num_classes = 23
models.fc = torch.nn.Linear(models.fc.in_features, num_classes)
models.load_state_dict(torch.load(path_model))

optimizers = optim.SGD(models.parameters(), lr=0.001, momentum=0.9)

In [ ]:
models.eval()
outputs_pred = []
outputs_true = []
with torch.no_grad(): 
    test_error_count = 0.0
    for images, labels in iter(test_set_method2):
        outputs = models(images)
        outputs_true.append(labels)
        outputs_pred.append(outputs.argmax(1))
        test_error_count += float(len(labels[labels != outputs.argmax(1)]))
    test_accuracy = 1.0 - float(test_error_count) / float(len(test_set_method2)*batch_size)
    print('accuracy test set: %f' % (test_accuracy))

In [ ]:
confusion_matrix_bis(outputs_true, outputs_pred)

Testing on new split

In [ ]:
path_model = r'best_model_optmiSGD_lossCrossEntrop_dav_basic_with_mom.pth'
models = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_resnet50', pretrained=True)
num_classes, batch_size = 23, 25
models.fc = torch.nn.Linear(models.fc.in_features, num_classes)
models.load_state_dict(torch.load(path_model))

optimizers = optim.SGD(models.parameters(), lr=0.001)

In [ ]:
models.eval()
outputs_pred = []
outputs_true = []
with torch.no_grad(): 
    test_error_count = 0.0
    for images, labels in iter(test_dataset_unnormalized):
        outputs = models(images)
        outputs_true.append(labels)
        outputs_pred.append(outputs.argmax(1))
        test_error_count += float(len(labels[labels != outputs.argmax(1)]))
    test_accuracy = 1.0 - float(test_error_count) / float(len(test_dataset_unnormalized)*batch_size)
    print('accuracy test set: %f' % (test_accuracy))

In [ ]:
confusion_matrix_bis(outputs_true, outputs_pred)

# Method 3

Testing on data loader

In [ ]:
path_model = r''
models = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_resnet50', pretrained=True)
num_classes, batch_size = 23, 25
models.fc = torch.nn.Linear(models.fc.in_features, num_classes)
models.load_state_dict(torch.load(path_model))

optimizers = optim.SGD(models.parameters(), lr=0.001, momentum=0.9)

In [ ]:
models.eval()
outputs_pred = []
outputs_true = []
with torch.no_grad(): 
    test_error_count = 0.0
    for images, labels in iter(test_set_method3):
        outputs = models(images)
        outputs_true.append(labels)
        outputs_pred.append(outputs.argmax(1))
        test_error_count += float(len(labels[labels != outputs.argmax(1)]))
    test_accuracy = 1.0 - float(test_error_count) / float(len(test_set_method3)*batch_size)
    print('accuracy test set: %f' % (test_accuracy))

In [ ]:
confusion_matrix_bis(outputs_true, outputs_pred)

Testing on new split

In [ ]:
path_model = r''
models = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_resnet50', pretrained=True)
num_classes, batch_size = 23, 25
models.fc = torch.nn.Linear(models.fc.in_features, num_classes)
models.load_state_dict(torch.load(path_model))

optimizers = optim.SGD(models.parameters(), lr=0.001, momentum=0.9)

In [ ]:
models.eval()
outputs_pred = []
outputs_true = []
with torch.no_grad(): 
    test_error_count = 0.0
    for images, labels in iter(test_dataset_normalized):
        outputs = models(images)
        outputs_true.append(labels)
        outputs_pred.append(outputs.argmax(1))
        test_error_count += float(len(labels[labels != outputs.argmax(1)]))
    test_accuracy = 1.0 - float(test_error_count) / float(len(test_dataset_normalized)*batch_size)
    print('accuracy test set: %f' % (test_accuracy))

In [ ]:
confusion_matrix_bis(outputs_true, outputs_pred)

# Method 4

Testing on data loader

In [ ]:
path_model = r'./fixing_issues/best_model_optmiSGD_lossCrossEntrop_dav_lay6and7.pth'
models = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_resnet50', pretrained=True)
num_classes, batch_size = 23, 25
models.fc = torch.nn.Linear(models.fc.in_features, num_classes)
models.load_state_dict(torch.load(path_model))

optimizers = optim.SGD(models.parameters(), lr=0.001, momentum=0.9)

In [ ]:
models.eval()
outputs_pred = []
outputs_true = []
with torch.no_grad(): 
    test_error_count = 0.0
    for images, labels in iter(test_set_method4):
        outputs = models(images)
        outputs_true.append(labels)
        outputs_pred.append(outputs.argmax(1))
        test_error_count += float(len(labels[labels != outputs.argmax(1)]))
    test_accuracy = 1.0 - float(test_error_count) / float(len(test_set_method4)*batch_size)
    print('accuracy test set: %f' % (test_accuracy))

In [ ]:
confusion_matrix_bis(outputs_true, outputs_pred)

Testing on new split

In [ ]:
path_model = r'./fixing_issues/best_model_optmiSGD_lossCrossEntrop_dav_lay6and7.pth'
models = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_resnet50', pretrained=True)
num_classes, batch_size = 23, 25
models.fc = torch.nn.Linear(models.fc.in_features, num_classes)
models.load_state_dict(torch.load(path_model))

optimizers = optim.SGD(models.parameters(), lr=0.001, momentum=0.9)

In [ ]:
models.eval()
outputs_pred = []
outputs_true = []
with torch.no_grad(): 
    test_error_count = 0.0
    for images, labels in iter(test_dataset_normalized):
        outputs = models(images)
        outputs_true.append(labels)
        outputs_pred.append(outputs.argmax(1))
        test_error_count += float(len(labels[labels != outputs.argmax(1)]))
    test_accuracy = 1.0 - float(test_error_count) / float(len(test_dataset_normalized)*batch_size)
    print('accuracy test set: %f' % (test_accuracy))

In [ ]:
confusion_matrix_bis(outputs_true, outputs_pred)

# Method 5

Testing on data loader

In [ ]:
path_model = r'./fixing_issues/best_model_optmiSGD_lossCrossEntrop_dav_w_2_transf.pth'
models = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_resnet50', pretrained=True)
num_classes, batch_size = 23, 25
models.fc = torch.nn.Linear(models.fc.in_features, num_classes)
models.load_state_dict(torch.load(path_model))

optimizers = optim.SGD(models.parameters(), lr=0.001, momentum=0.9)

In [ ]:
models.eval()
outputs_pred = []
outputs_true = []
with torch.no_grad(): 
    test_error_count = 0.0
    for images, labels in iter(test_set_method5):
        outputs = models(images)
        outputs_true.append(labels)
        outputs_pred.append(outputs.argmax(1))
        test_error_count += float(len(labels[labels != outputs.argmax(1)]))
    test_accuracy = 1.0 - float(test_error_count) / float(len(test_set_method5)*batch_size)
    print('accuracy test set: %f' % (test_accuracy))

In [ ]:
confusion_matrix_bis(outputs_true, outputs_pred)

Testing on new split

In [ ]:
path_model = r'./fixing_issues/best_model_optmiSGD_lossCrossEntrop_dav_w_2_transf.pth'
models = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_resnet50', pretrained=True)
num_classes, batch_size = 23, 25
models.fc = torch.nn.Linear(models.fc.in_features, num_classes)
models.load_state_dict(torch.load(path_model))

optimizers = optim.SGD(models.parameters(), lr=0.001, momentum=0.9)

In [ ]:
models.eval()
outputs_pred = []
outputs_true = []
with torch.no_grad(): 
    test_error_count = 0.0
    for images, labels in iter(test_dataset_normalized):
        outputs = models(images)
        outputs_true.append(labels)
        outputs_pred.append(outputs.argmax(1))
        test_error_count += float(len(labels[labels != outputs.argmax(1)]))
    test_accuracy = 1.0 - float(test_error_count) / float(len(test_dataset_normalized)*batch_size)
    print('accuracy test set: %f' % (test_accuracy))

In [ ]:
confusion_matrix_bis(outputs_true, outputs_pred)

# Method 6

Testing on data loader

In [ ]:
path_model = r'./fixing_issues/best_model_optmiSGD_lossCrossEntrop_dav_w_transf.pth'
models = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_resnet50', pretrained=True)
num_classes, batch_size = 23, 25
models.fc = torch.nn.Linear(models.fc.in_features, num_classes)
models.load_state_dict(torch.load(path_model))

optimizers = optim.SGD(models.parameters(), lr=0.001, momentum=0.9)

In [ ]:
models.eval()
outputs_pred = []
outputs_true = []
with torch.no_grad(): 
    test_error_count = 0.0
    for images, labels in iter(test_set_method6):
        outputs = models(images)
        outputs_true.append(labels)
        outputs_pred.append(outputs.argmax(1))
        test_error_count += float(len(labels[labels != outputs.argmax(1)]))
    test_accuracy = 1.0 - float(test_error_count) / float(len(test_set_method6)*batch_size)
    print('accuracy test set: %f' % (test_accuracy))

In [ ]:
confusion_matrix_bis(outputs_true, outputs_pred)

Testing on new split

In [ ]:
path_model = r'./fixing_issues/best_model_optmiSGD_lossCrossEntrop_dav_w_transf.pth'
models = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_resnet50', pretrained=True)
num_classes, batch_size = 23, 25
models.fc = torch.nn.Linear(models.fc.in_features, num_classes)
models.load_state_dict(torch.load(path_model))

optimizers = optim.SGD(models.parameters(), lr=0.001, momentum=0.9)

In [ ]:
models.eval()
outputs_pred = []
outputs_true = []
with torch.no_grad(): 
    test_error_count = 0.0
    for images, labels in iter(test_dataset_normalized):
        outputs = models(images)
        outputs_true.append(labels)
        outputs_pred.append(outputs.argmax(1))
        test_error_count += float(len(labels[labels != outputs.argmax(1)]))
    test_accuracy = 1.0 - float(test_error_count) / float(len(test_dataset_normalized)*batch_size)
    print('accuracy test set: %f' % (test_accuracy))

In [ ]:
confusion_matrix_bis(outputs_true, outputs_pred)